In [1]:
from fairseq.models.bart import BARTModel
import torch
from rouge import Rouge, FilesRouge
from tqdm import tqdm_notebook as tqdm
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("gpu num: ", n_gpu)

gpu num:  1


In [2]:
bart = BARTModel.from_pretrained(
    'path/to/checkpint',
    checkpoint_file='checkpoint_best.pt',
    data_name_or_path='path/to/data-bin-file'
)

In [3]:
bart.eval()
bart.cuda()

BARTHubInterface(
  (model): BARTModel(
    (encoder): TransformerEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
        (1): TransformerEnc

In [4]:
from tqdm import tqdm_notebook as tqdm

In [5]:
count = 1
bsz = 8
with open('../data/test_sent_trans_cons_label.source') as source, open('../data/test_sent_c99_label.source') as source2, open('./test_best_multi_attn_best.hypo', 'wt', encoding='utf-8') as fout:
    s1 = source.readlines()
    s2 = source2.readlines()

    slines = [s1[0].strip()]
    slines2 = [s2[0].strip()]

    for i in tqdm(range(1, len(s1))):
        if count % bsz == 0:
            with torch.no_grad():
                if args.multi_views:
                    hypotheses_batch = bart.sample(slines, sentences2 = slines2, balance = True, beam=4, lenpen=2.0, max_len_b=100, min_len=5, no_repeat_ngram_size=3)
                else:
                    hypotheses_batch = bart.sample(slines, beam=4, lenpen=2.0, max_len_b=100, min_len=5, no_repeat_ngram_size=3)
            for hypothesis in hypotheses_batch:
                fout.write(hypothesis + '\n')
                fout.flush()
            slines = []
            slines2 = []

        slines.append(s1[i].strip())
        slines2.append(s2[i].strip())

        count += 1

    if slines != []:
        if args.multi_views:
            hypotheses_batch = bart.sample(slines, sentences2 = slines2, balance = True, beam=4, lenpen=2.0, max_len_b=100, min_len=5, no_repeat_ngram_size=3)
        else:
            hypotheses_batch = bart.sample(slines, beam=4, lenpen=2.0, max_len_b=100, min_len=5, no_repeat_ngram_size=3)

        for hypothesis in hypotheses_batch:
            fout.write(hypothesis + '\n')
            fout.flush()
            
hyp_path = './test_best_multi_attn_best_.hypo'
ref_path = '../data/test_sent_trans_cons_label.target'
hypothesis = []
with open(hyp_path, 'r') as f:
    lines = f.readlines()
    for l in lines:
        hypothesis.append(l[:-1])

reference = []
with open(ref_path, 'r') as f:
    lines = f.readlines()
    for l in lines:
        reference.append(l[:-1])

rouge = Rouge()
print('Test', rouge.get_scores(hypothesis, reference, avg = True))
        

/nethome/jchen896/anaconda3/envs/new_torch/lib/python3.6/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  
